In [1]:
# 3. Import libraries and modules
import numpy as np
import pandas as pd
np.random.seed(123)  # for reproducibility

from os import listdir
from os.path import isfile, join

import scipy.io
from PIL import Image

from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D, MaxPooling2D
from keras.utils import np_utils
from keras.datasets import mnist

from keras.preprocessing.image import load_img, img_to_array
from keras.applications.vgg16 import preprocess_input


Using Theano backend.


In [2]:
# Method to read picture information from a matlab file
def load_img_from_mat(mat_file):

    mat_data = scipy.io.loadmat(mat_file)
    # load annotations for each picture
    dogs_annotation = []
    
    for img_annotation in mat_data['annotation_list']:
        dog_annotation = str(img_annotation[0][0])
        dogs_annotation.append(dog_annotation)

    # load picture filenames and path for each picture contained in mat file
    dogs_full_path = []
    dogs_folder = []
    dogs_file = []

    for img_file in mat_data['file_list']:
        dog_full_path = str(img_file[0][0])
        folder, filen =  dog_full_path.split('/')
        dogs_full_path.append(dog_full_path)
        dogs_folder.append(folder)
        dogs_file.append(filen)

    # Create dataframe with these informations
    file_df = pd.DataFrame({'file' : dogs_file,
                             'folder' : dogs_folder,
                             'full_path' : dogs_full_path,
                             'annotation' : dogs_annotation})
    
    # Load label information and create a complete dataframe
    labels_df = pd.DataFrame(mat_data['labels'], columns = ['label'])
    file_df = file_df.join(labels_df)
    
    return file_df

In [3]:
# Method to read a file picture and return it as VGG-16 requires
def load_img_file(variant, img_folder, img_file):
    picture_file = join(variant, img_folder, img_file)

    img = load_img(picture_file, target_size=(224, 224))  # Charger l'image
    img = img_to_array(img)  # Convertir en tableau numpy
    #img = img.reshape((1, img.shape[0], img.shape[1], img.shape[2]))  # Créer la collection d'images (un seul échantillon)
    img = preprocess_input(img)  # Prétraiter l'image comme le veut VGG-16
    
    return img

In [4]:
# Create picture array
def get_pict_array(pict_df):
    # Variable to hold all pictures
    X_pict_array = []
    first = True
    for dog_folder, dog_file in np.array(pict_df[['folder', 'file']]):
        img = load_img_file(bw, dog_folder, dog_file)
        if first:
            X_pict_array = [img]
            first = False
        else:
            X_pict_array += [img]
            
    return np.array(X_pict_array)

In [5]:
# Define data directories
my_data = "../data"
images = my_data + '/Images' # Images folder
bw = my_data + '/BWEqu'

In [6]:
# Parameters

# Number of dogs breeds to include
nb_breeds = 2

In [7]:
# Load train data
train_df = load_img_from_mat(my_data + '/train_list.mat')
sample_train_df = train_df[train_df['label'] <= nb_breeds]

X_train = get_pict_array(sample_train_df)
Y_train = sample_train_df['label'] - 1

# Load test data
test_df = load_img_from_mat(my_data + '/test_list.mat')
sample_test_df = test_df[test_df['label'] <= nb_breeds]

X_test = get_pict_array(sample_test_df)
Y_test = sample_test_df['label'] - 1

In [8]:
X_train.shape


(200, 3, 224, 224)

In [9]:
# 5. Preprocess input data
#X_train = X_train.reshape(X_train.shape[0], 1, 224, 224)

X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train /= 255
X_test /= 255
 

In [10]:
# 6. Preprocess class labels
Y_train = np_utils.to_categorical(Y_train, nb_breeds)
Y_test = np_utils.to_categorical(Y_test, nb_breeds)
 

In [11]:
# 7. Define model architecture
model = Sequential()
 
model.add(Convolution2D(32, (3, 3), activation='relu', input_shape=(3,224,224)))
model.add(Convolution2D(32, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))
 
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(nb_breeds, activation='softmax'))
 

In [12]:
# 8. Compile model
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
 

In [13]:
# 9. Fit model on training data
model.fit(X_train, Y_train, 
          batch_size=32, epochs=10, verbose=1)
 

Epoch 1/10
200/200 [==============================] - 23s 114ms/step - loss: 3.6850 - acc: 0.5200
Epoch 2/10
200/200 [==============================] - 22s 112ms/step - loss: 0.7295 - acc: 0.5450
Epoch 3/10
200/200 [==============================] - 22s 111ms/step - loss: 0.5599 - acc: 0.7850
Epoch 4/10
200/200 [==============================] - 22s 111ms/step - loss: 0.3957 - acc: 0.8250
Epoch 5/10
200/200 [==============================] - 22s 110ms/step - loss: 0.2864 - acc: 0.8800
Epoch 6/10
200/200 [==============================] - 22s 112ms/step - loss: 0.1346 - acc: 0.9700
Epoch 7/10
200/200 [==============================] - 22s 111ms/step - loss: 0.1579 - acc: 0.9450
Epoch 8/10
200/200 [==============================] - 22s 110ms/step - loss: 0.0618 - acc: 1.0000
Epoch 9/10
200/200 [==============================] - 22s 111ms/step - loss: 0.0461 - acc: 0.9950
Epoch 10/10
200/200 [==============================] - 22s 112ms/step - loss: 0.0229 - acc: 1.0000


In [14]:
# 10. Evaluate model on test data
score = model.evaluate(X_test, Y_test, verbose=0)

In [15]:
score


[0.5930086639675781, 0.7591240888964521]

In [16]:
model.metrics_names

['loss', 'acc']